In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chardet
from collections import Counter
import string
import nltk
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

In [2]:
# Read Input Data
def read_mycsv(path):
    with open(path, 'rb') as f:
        result = chardet.detect(f.read())
    return pd.read_csv(path, encoding=result['encoding'])

In [3]:
#data = read_mycsv("CDNPv3_Qs.csv")
data = read_mycsv("CDNP_MostFreqv1.csv")
data.head()

,Id,Comment1,Comment1 Keywords,Comment2,Comment2 Keywords,Comment3,Comment3 Keywords,Comment4,Comment4 Keywords,Strategy,Strategy Keywords
0,0,block that bounders and st where 3 storeys are...,"['storey density', 'block', 'storeys', 'storey...","Nicholson, and streets should be capped at two...","['streets', 'Nicholson', 'storeys']",the 174 and 175 bus services with a view to up...,"['primary local road', 'additional density', '...",of the suburbs best attributes is the bike way...,"['asset', 'suburbs best attributes', 'bike way...",existing residents amenity and the character o...,"['rate inner city suburb', 'ridiculous approva..."
1,1,rapid development of with large apartment buil...,"['suburb', 'traffic lights', 'approval', 'larg...",I strongly agree that development heights with...,"['hospital car park', 'streets', 'development ...",access to and between areas of is currently we...,"['access', 'areas of Greenslopes', 'road', 'bu...",main reason people choose to live in is becaus...,"['low density', 'increased density', 'density ...",4 comments.,['comments']
2,2,I think large apartment developments should be...,"['Street', 'large parks', 'large apartment dev...","RE expansion - a town square, visible from the...","['lively space', 'shady space', 'boost busines...",I strongly disagree that should be utilised as...,"['priority access road', 'residential street',...",I LOVE THESE PLANS! important. I also agree wi...,"['shady boulevarde', 'walking', 'enjoyable opt...",I support some areas as you can see by my subm...,"['residential street', 'priority access street..."
3,3,I believe that allowing 5 storey unit developm...,"['current residential area', 'residential stre...",space (not concrete space with trees) should b...,"['concrete space', 'trees', 'developments']",A direct access path from the to the other sid...,"['areas', 'direct access path', 'connection bu...","I would like our ""green levy"" on rates to be u...","['fewer green areas', 'green levy', 'adequate ...","I'm in full support of the green plans, effort...","['neighbourhood character', 'area', 'efforts',..."
4,4,our logic is that prehouses with gardens is th...,"['neighbourhood character', 'character residen...",i believe some development around the hospital...,"['development', 'hospital', 'community', 'sens...","paths, public transport and pedestrian access ...","['pedestrian access', 'big priority', 'public ...",seems increased density (up to 6 storeys in so...,"['CB Park', 'increased density', 'OPEN SPACE',...",all,[]


In [22]:
data = read_mycsv("Comment44.csv")
data.head()

,Id,Keyword
0,0,"['asset', 'suburbs best attributes', 'bike way..."
1,1,"['low density', 'increased density', 'density ..."
2,2,"['shady boulevarde', 'walking', 'enjoyable opt..."
3,3,"['fewer green areas', 'green levy', 'adequate ..."
4,4,"['CB Park', 'increased density', 'OPEN SPACE',..."


In [16]:
def comment_raiz(comment):
    text = []
    for lista in comment:
        valids = [word for word in lista if word not in stopwords and word not in punctuation 
                  and len(word)>2]
        valids_true = [''.join([char for char in word if char not in punctuation]) for word in valids if 
                       len(''.join([char for char in word if char not in punctuation]))>0]
        text.append(valids_true)
    return text

In [17]:
def counter(comment_clear):
    cnt = Counter()
    for words in comment_clear:
        for word in words:
            cnt[word] += 1
    return cnt

In [18]:
stopwords = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation
punctuation += '0123456789'

In [23]:
dataS=data["Keyword"]
dataS1 = dataS.str.lower()
dataS2 = dataS1.values.tolist()
dataS3 = [nltk.tokenize.wordpunct_tokenize(text) for text in dataS2]
dataS4 = comment_raiz(dataS3)
dataS5 = counter(dataS4)
dataS5.most_common(60)

[('green', 63),
 ('development', 60),
 ('parks', 53),
 ('park', 48),
 ('space', 41),
 ('area', 37),
 ('areas', 31),
 ('storey', 25),
 ('density', 22),
 ('people', 19),
 ('new', 18),
 ('community', 17),
 ('building', 17),
 ('developments', 15),
 ('storeys', 15),
 ('mountain', 15),
 ('trees', 15),
 ('current', 14),
 ('plan', 14),
 ('heights', 14),
 ('coffee', 14),
 ('natural', 13),
 ('access', 13),
 ('residential', 12),
 ('existing', 12),
 ('way', 11),
 ('increased', 11),
 ('streets', 11),
 ('high', 11),
 ('residents', 11),
 ('facilities', 11),
 ('local', 11),
 ('dog', 11),
 ('city', 10),
 ('road', 10),
 ('medium', 10),
 ('plans', 10),
 ('open', 10),
 ('shops', 10),
 ('low', 9),
 ('traffic', 9),
 ('spaces', 9),
 ('character', 9),
 ('buildings', 9),
 ('land', 9),
 ('street', 8),
 ('hospital', 8),
 ('leafy', 8),
 ('proposed', 8),
 ('council', 8),
 ('population', 8),
 ('things', 8),
 ('commercial', 8),
 ('appropriate', 8),
 ('suburbs', 7),
 ('number', 7),
 ('parking', 7),
 ('estate', 7),
 (